# Exploratory Analysis of Phone Prices

In [25]:
# import relevant libraries
import pandas as pd

In [26]:
# execute the following statement after downloading the kaggle.json file as per the instructions at https://saturncloud.io/blog/how-to-import-kaggle-datasets-into-jupyter-notebook/
!mv /tmp/kaggle.json $HOME/.kaggle/

mv: cannot stat '/tmp/kaggle.json': No such file or directory


In [27]:
import kaggle
# ignore the warnings if any, if you are running it on the personal computer

In [28]:
DATA_DIR = './data/'

In [29]:
dataset_name = 'berkayeserr/phone-prices'
kaggle.api.dataset_download_files(dataset_name, path=DATA_DIR, unzip=True)

In [31]:
df = pd.read_csv(DATA_DIR+'cleaned_all_phones.csv')
df.head

<bound method NDFrame.head of             phone_name   brand           os  inches resolution  battery  \
0         Y6II Compact  Huawei  Android 5.1    5.00   720x1280     2200   
1             K20 plus      LG  Android 7.0    5.30   720x1280     2700   
2       P8 Lite (2017)  Huawei  Android 7.0    5.20  1080x1920     3000   
3         Redmi Note 4  Xiaomi  Android 6.0    5.50  1080x1920     4100   
4                  P10  Huawei  Android 7.0    5.10  1080x1920     3200   
...                ...     ...          ...     ...        ...      ...   
1507         vivo Y77t    Vivo   Android 13    6.64  1080x2388     5000   
1508               11x  Realme   Android 13    6.72  1080x2400     5000   
1509               GT5  Realme   Android 13    6.74  1240x2772     5240   
1510          GT5 240W  Realme   Android 13    6.74  1240x2772     4600   
1511  vivo iQOO Z7 Pro    Vivo   Android 13    6.78  1080x2400     4600   

     battery_type  ram(GB) announcement_date  weight(g)  ...  video_1